# Exercises

Use your new JSON skills to answer these questions:

1. The earthquakes JSON has a key `tsunami` that's set to a value of `1` for large earthquakes in oceanic regions (though it doesn't mean a tsunami actually happened). Using either path or field extraction operators, find earthquakes with a `tsunami` value of `1` & include their location, time, & magnitude in your results.
2. Use the following `CREATE TABLE` statement to add the table `earthquakes_from_json` to your `analysis` database. Using field & path extraction operators, write an `INSERT` statement to fill the table with the correct value for each earthquake.

```
CREATE TABLE earthquakes_from_json (
    id text PRIMARY KEY,
    title text,
    type text,
    quake_date timestamp with time zone,
    mag numeric,
    place text,
    earthquake_point geography(POINT, 4326),
    url text
);
```

3. Try writing a query to generate the following JSON using the data in the `teachers` & `teachers_lab_access` tables from previous lessons. It's helpful to remember that the `teachers` table has a one-to-many reationships with `teachers_lab_access`; the first three keys must come from `teachers`, & the JSON objects in the array of `lab_access` will come from `teachers_lab_access`. Hint: you'll need to use a subquery in your `SELECT` list & a function called `json_agg()` to create the `lab_access` array.

```
{
 "id": 6,
 "fn": "Kathleen",
 "ln": "Roush",
 "lab_access": [{"lab_name": "Science B",
                 "access_time": "2022-12-17T16:00:00-05:00"},
                {"lab_name": "Science A",
                 "access_time": "2022-12-07T10:01:00-05:00"}]
}
```

---

# 1.

```
SELECT (earthquake #>> '{properties, place}') AS location,
	   to_timestamp((earthquake #>> '{properties, 
	   	   time}')::bigint / 1000) AT TIME ZONE 'UTC' 
		   AS time,
	   (earthquake #>> '{properties, mag}')::numeric
	   	   AS magnitude,
	   (earthquake #>> '{properties, tsunami}')::smallint
		   AS tsunami
FROM earthquakes
WHERE (earthquake #>> '{properties, tsunami}')::smallint = 1
ORDER BY magnitude DESC;
```

Here are the earthquakes that triggered tsunamis, ordered by descending magnitude.

<img src = "Exercise Images/Earthquakes That Triggered Tsunamis.png" width = "600" style = "margin:auto"/>

# 2. 

We'll start with a `SELECT` statement to see if we can pull all the values the `CREATE TABLE` statement is asking for.

```
SELECT id,
	   (earthquake #>> '{properties, title}') AS title,
	   (earthquake #>> '{properties, type}') AS type,
	   to_timestamp((earthquake #>> '{properties,
	   	   time}')::bigint / 1000) AS quake_date,
	   (earthquake #>> '{properties, mag}')::numeric 
	   	   AS magnitude,
	   (earthquake #>> '{properties, place}') AS place,
	   earthquake_point,
	   (earthquake #>> '{properties, detail}') AS url
FROM earthquakes
ORDER BY id;
```

<img src = "Exercise Images/Create earthquakes_from_json Table 1.png" width = "600" style = "margin:auto"/>

Looks ok to me. We'll add this into our `INSERT` statement & check if it worked correctly.

```
INSERT INTO earthquakes_from_json
SELECT id,
	   (earthquake #>> '{properties, title}') AS title,
	   (earthquake #>> '{properties, type}') AS type,
	   to_timestamp((earthquake #>> '{properties,
	   	   time}')::bigint / 1000) AS quake_date,
	   (earthquake #>> '{properties, mag}')::numeric 
	   	   AS magnitude,
	   (earthquake #>> '{properties, place}') AS place,
	   earthquake_point,
	   (earthquake #>> '{properties, detail}') AS url
FROM earthquakes
ORDER BY id;

SELECT * FROM earthquakes_from_json;
```

<img src = "Exercise Images/Create earthquakes_from_json Table 2.png" width = "600" style = "margin:auto"/>

It worked :)

# 3.

Let's remind ourselves what the `teachers` & `teachers_lab_access` tables look like. Here is the `teachers` table:

<img src = "Exercise Images/teachers Table.png" width = "600" style = "margin:auto"/>

Here is the `teachers_lab_access` table:

<img src = "Exercise Images/teachers_lab_access Table.png" width = "600" style = "margin:auto"/>

This one was messy, but I managed to make it work:

```
WITH tl 
AS (
	SELECT *
	FROM teachers AS t
	LEFT JOIN teachers_lab_access AS tla
		ON t.id = tla.teacher_id
	WHERE t.id = 6 AND tla.access_id != 4
	ORDER BY tla.access_time DESC
)
SELECT lab_exercise
FROM (
	SELECT id, first_name, last_name,
		   json_build_object(
		'id', id,
		'fn', first_name,
		'ln', last_name,
		'lab_access', (
			(SELECT json_agg(json_build_object(
				'lab_name', lab_name,
				'access_time', access_time
			)) AS lab_access
			FROM tl)
		)) AS lab_exercise
	FROM tl
	GROUP BY id, first_name, last_name
);
```

I've formatted the result:

<img src = "Exercise Images/Creating Json Object with Postgres.png" width = "600" style = "margin:auto"/>

There's probably a more efficient way to write this query.